In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [87]:
path = os.path.join('data/clean')

BATCH_SIZE = 16
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

train = image_dataset_from_directory(path, 
                                     seed=0,
                                     validation_split=0.2,
                                     shuffle=True, 
                                     batch_size=BATCH_SIZE, 
                                     image_size=IMG_SIZE, 
                                     subset="training",
                                     label_mode='categorical'
                                    )
val = image_dataset_from_directory(path, 
                                   seed=0,
                                   validation_split=0.2,
                                   shuffle=True, 
                                   batch_size=BATCH_SIZE, 
                                   image_size=IMG_SIZE, 
                                   subset="validation",
                                   label_mode='categorical'
                                  )

val_batches = tf.data.experimental.cardinality(val)
test = val.take(val_batches // 10)
val = val.skip(val_batches // 10)

Found 9937 files belonging to 100 classes.
Using 7950 files for training.
Found 9937 files belonging to 100 classes.
Using 1987 files for validation.


In [70]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.05),
])

preprocess_input = tf.keras.applications.vgg19.preprocess_input

In [71]:
base_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SHAPE,
    classes=len(train.class_names),
)

base_model.trainable = False

In [73]:
base_model.summary()

In [95]:
fc_layer = tf.keras.layers.Flatten()
dense_layer = tf.keras.layers.Dense(2048, activation='relu')
prediction_layer = tf.keras.layers.Dense(len(train.class_names), activation='softmax')

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = fc_layer(x)
x = dense_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

lr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['categorical_crossentropy'])

In [96]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_10 (Sequential)   (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice_3  [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_3 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 2048)             

In [97]:
tf.config.run_functions_eagerly(True)

In [98]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(val)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

  3/113 [..............................] - ETA: 10:04 - loss: 0.6989 - categorical_crossentropy: 13.6750

KeyboardInterrupt: 